In [ ]:
import tushare as ts
import os
import pandas as pd
import numpy as np
import math
import datetime
import time
import matplotlib.pyplot as plt
from matplotlib import cm
import requests
import matplotlib as mpl
from scipy.interpolate import make_interp_spline
from sklearn.utils.multiclass import type_of_target
ts.set_token('4264bdda6fbb66c87e086cfadaecdc887566b8c689f000965db52f3e')
pro = ts.pro_api()

# Every year's trading days number

In [ ]:
df_tradeday = pro.trade_cal(exchange='', start_date='20110101', end_date='20201231')
df_tradeday.head()
df_count=df_tradeday.groupby(df_tradeday['cal_date'].apply(lambda x:x[:4])).sum()

# Risk-free Rate (Shibor in %)

In [ ]:
os.chdir('Default_Data/')
os.chdir('Shibor/')

In [ ]:
rf_last = [] # Every year last date's risk-free rate
rf_mean = [] # Every year's mean risk-free rate
year= []
for file in os.listdir():
    print(file)
    data = pd.read_excel(file)
    year.append(file[7:11])
    rf_last.append(data.iloc[data.shape[0]-1,data.shape[1]-1])
    rf_mean.append(np.mean(data['1Y']))

In [ ]:
os.chdir('../')
os.chdir('../')

In [ ]:
rf_last

In [ ]:
year

# Match Stock ts_code

In [ ]:
# Get the bond issuers' name from the excel file
sheet = pd.read_excel('Bond_Issuer_ListCompornot.xlsx')
manually = pd.read_excel('manually.xlsx')

In [ ]:
# Basic information of list companies in SH and SZ market
basic_SHSZ = pro.stock_basic(exchange='', list_status='L', fields='ts_code,fullname')
# Basic information of list companies in HK market
basic_HK = pro.hk_basic(list_status='L', fields='ts_code,fullname')
# Merge them together
frames = [basic_SHSZ, basic_HK, manually]
basic = pd.concat(frames)

In [ ]:
# Get the stock codes
stockCode = sheet.merge(basic, on='fullname',how='left')
stockCode = pd.DataFrame(stockCode, columns=['fullname','ts_code'])
stockCode = stockCode.dropna(subset=['ts_code'])
stockCode.to_excel('stock_list.xlsx', index=False, encoding='utf_8_sig')

In [ ]:
def isnotin(a,b):
    if a in b:
        return False
    else:
        return True

In [ ]:
stockCode_SHSZ = stockCode[stockCode['ts_code'].map(lambda x:isnotin('.HK',x))]

In [ ]:
stockCode_SHSZ = stockCode_SHSZ['ts_code'].tolist()
stockCode_SHSZ = list(set(stockCode_SHSZ))

# 短期负债 长期负债 企业债券面值

In [ ]:
# Read CSMAR debt data
combas = pd.read_excel('FS_Combas.xlsx')

In [ ]:
combasData = combas.drop(combas[combas['Typrep'] == 'B'].index)
del combasData['Typrep']

In [ ]:
combasData.columns = ['Stkcd','end_date','current_debt','long_term_debt','non_current_debt','total_debt']

In [ ]:
dropList = []
for i in list(set(combasData['end_date'].tolist())):
    if i < '2011-12-31' or (i.endswith('12-31') == False):
        dropList.append(i)

In [ ]:
for i in dropList:
    combasData = combasData.drop(combasData[combasData['end_date'] == i].index)

In [ ]:
os.chdir('Default_Data/')
os.chdir('debt/')

In [ ]:
# Short-term debt, Long-term debt, Face value from Tushare
for i in stockCode_SHSZ:
    print(i)
    # Debt
    debtData = pro.balancesheet(ts_code=i, start_date='20110101', end_date='20210630', fields='end_date,total_cur_liab,total_ncl,total_liab')
    debtNew=debtData[debtData['end_date'].apply(lambda x:x[4:])=='1231']
    debt=debtNew.drop_duplicates()
    #debt.sort_values('end_date',inplace=True)
    debt.to_excel(i+'_debt.xlsx',index=False,encoding='utf_8_sig')

In [ ]:
os.chdir('../')
os.chdir('../')

# 每股净资产

In [ ]:
# Read CSMAR bps data
combas = pd.read_excel('FS_Combas.xlsx')
combasData = combas.drop(combas[combas['Typrep'] == 'B'].index)
del combasData['Typrep']
combasData.columns = ['Stkcd','end_date','current_debt','long_term_debt','non_current_debt','total_debt']
dropList = []
for i in list(set(combasData['end_date'].tolist())):
    if i < '2011-12-31' or (i.endswith('12-31') == False):
        dropList.append(i)
for i in dropList:
    combasData = combasData.drop(combasData[combasData['end_date'] == i].index)

In [ ]:
os.chdir('Default_Data/')
os.chdir('bps/')

In [ ]:
for i in stockCode_SHSZ:
    print(i)
    bpsData = pro.fina_indicator(ts_code=i, start_date='20110101', end_date='20210630', fields='ts_code,end_date,bps')
    d = dict(zip(bpsData['end_date'],bpsData['bps']))
    bpsData = []
    for j in d.keys():
        if j.endswith('1231') == True:
            bpsData.append([j,d[j]])
    bpsData = sorted(bpsData, key=lambda x:x[0], reverse=False)
    bps = pd.DataFrame(bpsData)
    bps.columns = ['date','bps']
    bps.to_excel(i+'_bps.xlsx',index=False,encoding='utf_8_sig')

In [ ]:
os.chdir('../')
os.chdir('../')

# 流通股与非流通股

In [ ]:
os.chdir('Default_Data/')
capchg = pd.read_csv('TRD_Capchg.csv')
os.chdir('../')
capchgData = pd.DataFrame(capchg, columns=['Stkcd','Shrchgdt','Nshrttl','Nshra'])
capchgData['Stkcd'] = pd.DataFrame([str(x).zfill(6) for x in capchgData['Stkcd'].tolist()])
stkcd = capchgData['Stkcd'].tolist()
for i in range(len(stkcd)):
    if stkcd[i][0] == '0' or stkcd[i][0] == '3':
        stkcd[i] = stkcd[i] + '.SZ'
    else:
        stkcd[i] = stkcd[i] + '.SH'
capchgData['Stkcd'] = pd.DataFrame(stkcd)
capchgData['Shrchgdt'] = pd.DataFrame([x.replace('-','') for x in capchgData['Shrchgdt'].tolist()])
lastDate = [x+'1231' for x in year]

# Weekly Stock Close Price 使用前复权后的周收盘价

In [ ]:
os.chdir('Default_Data/weekly_close/')

In [ ]:
for i in stockCode_SHSZ:
    print(i)
    closeData = ts.pro_bar(ts_code=i, freq='W', adj='qfq', start_date='20110101', end_date='20210101')
    closeData['ln_return'] = [math.log(closeData['close'][i]/closeData['pre_close'][i]) for i in range(len(closeData))]
    closeData['weekly_return'] = [(closeData['close'][i] - closeData['pre_close'][i])/closeData['pre_close'][i] for i in range(len(closeData))]
    closeData['square_ln'] = [pow(i,2) for i in closeData['ln_return'].tolist()]
    closeData['square_weekly'] = [pow(i,2) for i in closeData['weekly_return'].tolist()]
    closeData.to_excel(i+'_weekly.xlsx', index=False, encoding='utf_8_sig')

In [ ]:
os.chdir('../../')

# Summary Codes to calculate the default probability

In [ ]:
i='000008.SZ'
debtData = pro.balancesheet(ts_code=i, start_date='20110101', end_date='20210630', fields='end_date,total_cur_liab,total_ncl,total_liab')
debtNew=debtData[debtData['end_date'].apply(lambda x:x[4:])=='1231']
debt=debtNew.drop_duplicates()
shortTerm = debt['total_cur_liab'].tolist()
longTerm = debt['total_ncl'].tolist()
nan = longTerm[-1]

In [ ]:
# function to calculate sigmaE
def sigmaE(n, squaresum, sumsquare):
    if n <= 1:
        sigmae = nan
    else:
        if (squaresum/(n-1)) - 1/(n*(n-1)*pow(sumsquare,2)) <= 0:
            sigmae = nan
        else:
            nominator = math.sqrt((squaresum/(n-1)) - 1/(n*(n-1)*pow(sumsquare,2)))
            denominator = math.sqrt(1/n)
            sigmae = nominator/denominator
    return sigmae

In [ ]:
#import package
from scipy.optimize import fsolve
import math
from scipy import stats
#def exp sqrt
def st(x):
    x=float(x)
    return math.exp(-(x**2)/2)
#def normal distribution
#def N(x):
#    from scipy import integrate
#    nx,err=integrate.quad(st,-100,x)
#    nx=nx/math.sqrt(2*math.pi)
#    return nx

def N(x):
    nx=stats.norm.cdf(x)
    return nx

#second function

#def calculate the V and ThetaV
def f(Equity,SigmaE,D,r,t=1):
    Equity=float(Equity)
    D=float(D)
    r=float(r)
    if Equity==0 or SigmaE==0 or D==0:
        result=[0,0]
        return result
    else:
        def ff(x):

            x0=float(x[0])
            x1=float(x[1])
            d1=(math.log(abs(x0))-math.log(D)+(r+0.5*(x1**2))*t)/(x1*math.sqrt(t))
            d2=d1-x1*math.sqrt(t)
            return[
                Equity-x0*N(d1)+D*math.exp(-r*t)*N(d2),
                SigmaE-x0*x1*(N(d1)/Equity)
            ]
        result=fsolve(ff,[Equity,SigmaE])
        return result



In [ ]:
f(141276427,0.2893,1.25e8,0.0225,1)

# Tushare and CSMAR ln and weekly return with rf_last (Don't use anymore)

In [ ]:
os.chdir('Default_Data/output_ln_return/')

In [ ]:
# Data from Tushare
togetherFinal = []
togetherWeeklyFinal = []
for i in stockCode_SHSZ:
    print(i)
    # Debt
    debtData = pro.balancesheet(ts_code=i, start_date='20110101', end_date='20210630', fields='end_date,total_cur_liab,total_ncl,total_liab')
    debtNew=debtData[debtData['end_date'].apply(lambda x:x[4:])=='1231']
    debt=debtNew.drop_duplicates()
    shortTerm = debt['total_cur_liab'].tolist()
    longTerm = debt['total_ncl'].tolist()
    totalDebt = debt['total_liab'].tolist()
    endDate = debt['end_date'].tolist()
    
    # Calculate three kinds of DP
    dp = [] # DP
    dp_50 = [] # DP50
    dp_75 = [] # DP75
    d_facevalue = [] # D
    for y in year:
        for e in endDate:
            if e.startswith(y) == True:
                current = shortTerm[endDate.index(e)]
                long = longTerm[endDate.index(e)]
                if (current == None) or (long == None):
                    dp.append(nan)
                    dp_50.append(nan)
                    dp_75.append(nan)
                else:
                    dp.append(current)
                    dp_50.append(current+0.5*long)
                    dp_75.append(current+0.75*long)
                d_facevalue.append(totalDebt[endDate.index(e)])
            else:
                pass
    dpFrame = pd.DataFrame([lastDate,dp,dp_50,dp_75,d_facevalue]).T
    dpFrame.columns = ['end_date','DP','DP50','DP75','D']
    
    
    # BPS
    bpsData = pro.fina_indicator(ts_code=i, start_date='20110101', end_date='20210630', fields='ts_code,end_date,bps')
    d = dict(zip(bpsData['end_date'],bpsData['bps']))
    bpsData = []
    for j in d.keys():
        if j.endswith('1231') == True:
            bpsData.append([j,d[j]])
    bpsData = sorted(bpsData, key=lambda x:x[0], reverse=False)
    bps = pd.DataFrame(bpsData)
    bps.columns = ['end_date','bps']
    bpsData = bps['bps'].tolist()

    # Weekly Stock Return
    closeData = ts.pro_bar(ts_code=i, freq='W', adj='qfq', start_date='20110101', end_date='20210101')
    #closeData = pro.weekly(ts_code=i, start_date='20110101', end_date='20210101', fields='trade_date,close,pre_close')
    if type(closeData) == pd.DataFrame:
        closeData['ln_return'] = [math.log(closeData['close'][i]/closeData['pre_close'][i]) for i in range(len(closeData))]
        closeData['weekly_return'] = [(closeData['close'][i] - closeData['pre_close'][i])/closeData['pre_close'][i] for i in range(len(closeData))]
        closeData['square_ln'] = [pow(i,2) for i in closeData['ln_return'].tolist()]
        closeData['square_weekly'] = [pow(i,2) for i in closeData['weekly_return'].tolist()]
        closePrice = closeData['close'].tolist()
        lnList = closeData['ln_return'].tolist()
        weeklyList = closeData['weekly_return'].tolist()    
        squareLnList = closeData['square_ln'].tolist()
        squareWeeklyList = closeData['square_weekly'].tolist()
        tradeDate = closeData['trade_date'].tolist()
    else:
        pass
  
    
    # Start to calculate KMV
    sum_ln = []
    sum_weekly = []
    square_sum_ln = []
    square_sum_weekly = []
    mean_close = []
    nList = []
    for j in year:
        #print(j)
        sumln = []
        sumweekly = []
        squaresumln = []
        squaresumweekly = []
        meanClosePrice = []
        for k in tradeDate:
            if k.startswith(j) == True:
                sumln.append(lnList[tradeDate.index(k)])
                sumweekly.append(weeklyList[tradeDate.index(k)])
                squaresumln.append(squareLnList[tradeDate.index(k)])
                squaresumweekly.append(squareWeeklyList[tradeDate.index(k)])
                meanClosePrice.append(closePrice[tradeDate.index(k)])
            else:
                pass
        sum_ln.append(np.sum(sumln))
        sum_weekly.append(np.sum(sumweekly))        
        square_sum_ln.append(np.sum(squaresumln))
        square_sum_weekly.append(np.sum(squaresumweekly))
        if len(meanClosePrice) == 0:
            mean_close.append(0)
        else:
            mean_close.append(np.mean(meanClosePrice))
        nList.append(len(sumln))
    print('Done')
    weeklyReturn = pd.DataFrame([lastDate, sum_ln, sum_weekly, square_sum_ln, square_sum_weekly, mean_close, nList]).T
    weeklyReturn.columns = ['end_date','ln_return','weekly_return','square_ln_return','square_weekly_return','mean_close','numOfWeeks']
    
    # Calculate E
    # First, get the number of tradable shares and non-tradable shares
    shareData = capchgData.loc[capchgData['Stkcd'] == i]
    tradeShare = []
    nonTradeShare = []
    for l in lastDate:
        if l <= shareData['Shrchgdt'].tolist()[0]:
            total = 0
            trade = 0
        else:
            total = shareData.loc[shareData['Shrchgdt'] <= l]['Nshrttl'].tolist()[-1]
            trade = shareData.loc[shareData['Shrchgdt'] <= l]['Nshra'].tolist()[-1]
        nonTrade = total - trade
        tradeShare.append(trade)
        nonTradeShare.append(nonTrade)
    # Then calculate E
    equity = [] # E
    for r in range(len(bps)):
        tradeEquity = mean_close[r] * tradeShare[r]
        nonTradeEquity = bpsData[r] * nonTradeShare[r]
        equityMktVal = tradeEquity + nonTradeEquity
        equity.append(equityMktVal)
        
    # Use ln_Return to calculate sigmaE
    sigma_E_ln = []
    for ye in range(len(year)):
        sigma_E_ln.append(sigmaE(nList[ye], square_sum_ln[ye], sum_ln[ye]))
    eAndSigmaELn = pd.DataFrame([lastDate,equity,sigma_E_ln,rf_last]).T
    eAndSigmaELn.columns = ['end_date','E','sigmaE','r']
    
    totalDf = dpFrame.merge(debt, on='end_date',how='left')
    totalDf = totalDf.merge(bps, on='end_date',how='left')
    totalDf = totalDf.merge(weeklyReturn, on='end_date',how='left')
    totalDf = totalDf.merge(eAndSigmaELn, on='end_date',how='left')
    finalDf = totalDf.dropna(axis=0,how='any')
    

    output = []
    longth = finalDf.shape[0]
    compcode = []
    for ye in range(longth):
        compcode.append(i)
        Equity = finalDf['E'].tolist()[ye]
        SigmaE = finalDf['sigmaE'].tolist()[ye]
        D = finalDf['D'].tolist()[ye]
        r = finalDf['r'].tolist()[ye]/100
        medium =f(Equity,SigmaE,D,r,1)
        if medium[0] == 0 or medium[1] == 0:
            output.append('-')
        else:
            dd = (medium[0] - finalDf['DP50'].tolist()[ye])/(medium[0]*medium[1])
            output.append(1-N(dd))
    outputDf = pd.DataFrame([finalDf['end_date'].tolist(),output]).T
    outputDf.columns = ['end_date','default_probability']
    outputDf.to_excel(i+'.xlsx', index=False, encoding='utf_8_sig')
    
    if len(compcode) == 0:
        pass
    else:
        togetherOutput = pd.DataFrame([compcode, finalDf['end_date'].tolist(), output]).T
        togetherFinal.append(togetherOutput)
    
    
    

    # Use Weekly_Return to calculate sigmaE
    sigma_E_weekly = []
    for ye in range(len(year)):
        sigma_E_weekly.append(sigmaE(nList[ye], square_sum_weekly[ye], sum_weekly[ye]))
    eAndSigmaEweekly = pd.DataFrame([lastDate,equity,sigma_E_weekly,rf_last]).T
    eAndSigmaEweekly.columns = ['end_date','E','sigmaE','r']
    
    totalDfWeekly = dpFrame.merge(debt, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(bps, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(weeklyReturn, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(eAndSigmaEweekly, on='end_date',how='left')
    finalDfWeekly = totalDfWeekly.dropna(axis=0,how='any')
    

    outputWeekly = []
    longthWeekly = finalDfWeekly.shape[0]
    compcodeWeekly = []
    for ye in range(longthWeekly):
        compcodeWeekly.append(i)
        EquityWeekly = finalDfWeekly['E'].tolist()[ye]
        SigmaEWeekly = finalDfWeekly['sigmaE'].tolist()[ye]
        DWeekly = finalDfWeekly['D'].tolist()[ye]
        rWeekly = finalDfWeekly['r'].tolist()[ye]/100
        mediumWeekly = f(EquityWeekly,SigmaEWeekly,DWeekly,rWeekly,1)
        if mediumWeekly[0] == 0 or mediumWeekly[1] == 0:
            outputWeekly.append('-')
        else:
            dd = (mediumWeekly[0] - finalDfWeekly['DP50'].tolist()[ye])/(mediumWeekly[0]*mediumWeekly[1])
            outputWeekly.append(1-N(dd))
    outputDfWeekly = pd.DataFrame([finalDfWeekly['end_date'].tolist(),outputWeekly]).T
    outputDfWeekly.columns = ['end_date','default_probability']
    outputDfWeekly.to_excel(i+'_weekly.xlsx', index=False, encoding='utf_8_sig')
    
    if len(compcodeWeekly) == 0:
        pass
    else:
        togetherWeeklyOutput = pd.DataFrame([compcodeWeekly, finalDfWeekly['end_date'].tolist(), outputWeekly]).T
        togetherWeeklyFinal.append(togetherWeeklyOutput)
    
    

In [ ]:
os.chdir('../')

In [ ]:
outputTogether = togetherFinal
outputWeeklyTogether = togetherWeeklyFinal

In [ ]:
togetherDf = pd.concat(outputTogether)
togetherDf = togetherDf.drop_duplicates()
togetherDf.columns = ['compcode','end_date','default_probability']
togetherDf.to_excel('together_final.xlsx', index=False, encoding='utf_8_sig')

In [ ]:
togetherWeeklyDf = pd.concat(outputWeeklyTogether)
togetherWeeklyDf = togetherWeeklyDf.drop_duplicates()
togetherWeeklyDf.columns = ['compcode','end_date','default_probability']
togetherWeeklyDf.to_excel('together_weekly_return_final.xlsx', index=False, encoding='utf_8_sig')

In [ ]:
os.chdir('../')

In [ ]:
combasData

In [ ]:
os.chdir('Default_Data/output_ln_return_CSMAR/')

In [ ]:
# Data from CSMAR
togetherFinalCSMAR = []
togetherWeeklyFinalCSMAR = []
for i in stockCode_SHSZ:
    print(i)
    # Debt
    debtUse = combasData[combasData['Stkcd'] == i[:6]]
    shortTerm = debtUse['current_debt'].tolist()
    longTerm = debtUse['long_term_debt'].tolist()
    totalDebt = debtUse['total_debt'].tolist()
    endDate = [x.replace('-','') for x in debtUse['end_date'].tolist()]

    
    # Calculate three kinds of DP
    dp = [] # DP
    dp_50 = [] # DP50
    dp_75 = [] # DP75
    d_facevalue = [] # D
    for y in year:
        for e in endDate:
            if e.startswith(y) == True:
                current = shortTerm[endDate.index(e)]
                long = longTerm[endDate.index(e)]
                if (current == None) or (long == None):
                    dp.append(nan)
                    dp_50.append(nan)
                    dp_75.append(nan)
                else:
                    dp.append(current)
                    dp_50.append(current+0.5*long)
                    dp_75.append(current+0.75*long)
                d_facevalue.append(totalDebt[endDate.index(e)])
            else:
                pass
    dpFrame = pd.DataFrame([lastDate,dp,dp_50,dp_75,d_facevalue]).T
    dpFrame.columns = ['end_date','DP','DP50','DP75','D']
    
    
    # BPS
    bpsData = pro.fina_indicator(ts_code=i, start_date='20110101', end_date='20210630', fields='ts_code,end_date,bps')
    d = dict(zip(bpsData['end_date'],bpsData['bps']))
    bpsData = []
    for j in d.keys():
        if j.endswith('1231') == True:
            bpsData.append([j,d[j]])
    bpsData = sorted(bpsData, key=lambda x:x[0], reverse=False)
    bps = pd.DataFrame(bpsData)
    bps.columns = ['end_date','bps']
    bpsData = bps['bps'].tolist()

    # Weekly Stock Return
    closeData = ts.pro_bar(ts_code=i, freq='W', adj='qfq', start_date='20110101', end_date='20210101')
    #closeData = pro.weekly(ts_code=i, start_date='20110101', end_date='20210101', fields='trade_date,close,pre_close')
    if type(closeData) == pd.DataFrame:
        closeData['ln_return'] = [math.log(closeData['close'][i]/closeData['pre_close'][i]) for i in range(len(closeData))]
        closeData['weekly_return'] = [(closeData['close'][i] - closeData['pre_close'][i])/closeData['pre_close'][i] for i in range(len(closeData))]
        closeData['square_ln'] = [pow(i,2) for i in closeData['ln_return'].tolist()]
        closeData['square_weekly'] = [pow(i,2) for i in closeData['weekly_return'].tolist()]
        closePrice = closeData['close'].tolist()
        lnList = closeData['ln_return'].tolist()
        weeklyList = closeData['weekly_return'].tolist()    
        squareLnList = closeData['square_ln'].tolist()
        squareWeeklyList = closeData['square_weekly'].tolist()
        tradeDate = closeData['trade_date'].tolist()
    else:
        pass
  
    
    # Start to calculate KMV
    sum_ln = []
    sum_weekly = []
    square_sum_ln = []
    square_sum_weekly = []
    mean_close = []
    nList = []
    for j in year:
        #print(j)
        sumln = []
        sumweekly = []
        squaresumln = []
        squaresumweekly = []
        meanClosePrice = []
        for k in tradeDate:
            if k.startswith(j) == True:
                sumln.append(lnList[tradeDate.index(k)])
                sumweekly.append(weeklyList[tradeDate.index(k)])
                squaresumln.append(squareLnList[tradeDate.index(k)])
                squaresumweekly.append(squareWeeklyList[tradeDate.index(k)])
                meanClosePrice.append(closePrice[tradeDate.index(k)])
            else:
                pass
        sum_ln.append(np.sum(sumln))
        sum_weekly.append(np.sum(sumweekly))        
        square_sum_ln.append(np.sum(squaresumln))
        square_sum_weekly.append(np.sum(squaresumweekly))
        if len(meanClosePrice) == 0:
            mean_close.append(0)
        else:
            mean_close.append(np.mean(meanClosePrice))
        nList.append(len(sumln))
    print('Done')
    weeklyReturn = pd.DataFrame([lastDate, sum_ln, sum_weekly, square_sum_ln, square_sum_weekly, mean_close, nList]).T
    weeklyReturn.columns = ['end_date','ln_return','weekly_return','square_ln_return','square_weekly_return','mean_close','numOfWeeks']
    
    # Calculate E
    # First, get the number of tradable shares and non-tradable shares
    shareData = capchgData.loc[capchgData['Stkcd'] == i]
    tradeShare = []
    nonTradeShare = []
    for l in lastDate:
        if l <= shareData['Shrchgdt'].tolist()[0]:
            total = 0
            trade = 0
        else:
            total = shareData.loc[shareData['Shrchgdt'] <= l]['Nshrttl'].tolist()[-1]
            trade = shareData.loc[shareData['Shrchgdt'] <= l]['Nshra'].tolist()[-1]
        nonTrade = total - trade
        tradeShare.append(trade)
        nonTradeShare.append(nonTrade)
    # Then calculate E
    equity = [] # E
    for r in range(len(bps)):
        tradeEquity = mean_close[r] * tradeShare[r]
        nonTradeEquity = bpsData[r] * nonTradeShare[r]
        equityMktVal = tradeEquity + nonTradeEquity
        equity.append(equityMktVal)
        
    # Use ln_Return to calculate sigmaE
    sigma_E_ln = []
    for ye in range(len(year)):
        sigma_E_ln.append(sigmaE(nList[ye], square_sum_ln[ye], sum_ln[ye]))
    eAndSigmaELn = pd.DataFrame([lastDate,equity,sigma_E_ln,rf_last]).T
    eAndSigmaELn.columns = ['end_date','E','sigmaE','r']
    
    totalDf = dpFrame.merge(debt, on='end_date',how='left')
    totalDf = totalDf.merge(bps, on='end_date',how='left')
    totalDf = totalDf.merge(weeklyReturn, on='end_date',how='left')
    totalDf = totalDf.merge(eAndSigmaELn, on='end_date',how='left')
    finalDf = totalDf.dropna(axis=0,how='any')
    

    output = []
    longth = finalDf.shape[0]
    compcode = []
    for ye in range(longth):
        compcode.append(i)
        Equity = finalDf['E'].tolist()[ye]
        SigmaE = finalDf['sigmaE'].tolist()[ye]
        D = finalDf['D'].tolist()[ye]
        r = finalDf['r'].tolist()[ye]/100
        medium =f(Equity,SigmaE,D,r,1)
        if medium[0] == 0 or medium[1] == 0:
            output.append('-')
        else:
            dd = (medium[0] - finalDf['DP50'].tolist()[ye])/(medium[0]*medium[1])
            output.append(1-N(dd))
    outputDf = pd.DataFrame([finalDf['end_date'].tolist(),output]).T
    outputDf.columns = ['end_date','default_probability']
    outputDf.to_excel(i+'.xlsx', index=False, encoding='utf_8_sig')
    
    if len(compcode) == 0:
        pass
    else:
        togetherOutput = pd.DataFrame([compcode, finalDf['end_date'].tolist(), output]).T
        togetherFinalCSMAR.append(togetherOutput)
    
    
    
    
    

    # Use Weekly_Return to calculate sigmaE
    sigma_E_weekly = []
    for ye in range(len(year)):
        sigma_E_weekly.append(sigmaE(nList[ye], square_sum_weekly[ye], sum_weekly[ye]))
    eAndSigmaEweekly = pd.DataFrame([lastDate,equity,sigma_E_weekly,rf_last]).T
    eAndSigmaEweekly.columns = ['end_date','E','sigmaE','r']
    
    totalDfWeekly = dpFrame.merge(debt, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(bps, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(weeklyReturn, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(eAndSigmaEweekly, on='end_date',how='left')
    finalDfWeekly = totalDfWeekly.dropna(axis=0,how='any')
    

    outputWeekly = []
    longthWeekly = finalDfWeekly.shape[0]
    compcodeWeekly = []
    for ye in range(longthWeekly):
        compcodeWeekly.append(i)
        EquityWeekly = finalDfWeekly['E'].tolist()[ye]
        SigmaEWeekly = finalDfWeekly['sigmaE'].tolist()[ye]
        DWeekly = finalDfWeekly['D'].tolist()[ye]
        rWeekly = finalDfWeekly['r'].tolist()[ye]/100
        mediumWeekly = f(EquityWeekly,SigmaEWeekly,DWeekly,rWeekly,1)
        if mediumWeekly[0] == 0 or mediumWeekly[1] == 0:
            outputWeekly.append('-')
        else:
            dd = (mediumWeekly[0] - finalDfWeekly['DP50'].tolist()[ye])/(mediumWeekly[0]*mediumWeekly[1])
            outputWeekly.append(1-N(dd))
    outputDfWeekly = pd.DataFrame([finalDfWeekly['end_date'].tolist(),outputWeekly]).T
    outputDfWeekly.columns = ['end_date','default_probability']
    outputDfWeekly.to_excel(i+'_weekly.xlsx', index=False, encoding='utf_8_sig')
    
    if len(compcodeWeekly) == 0:
        pass
    else:
        togetherWeeklyOutput = pd.DataFrame([compcodeWeekly, finalDfWeekly['end_date'].tolist(), outputWeekly]).T
        togetherWeeklyFinalCSMAR.append(togetherWeeklyOutput)
    
    

In [ ]:
os.chdir('../')
outputTogetherCSMAR = togetherFinalCSMAR
outputWeeklyTogetherCSMAR = togetherWeeklyFinalCSMAR
togetherDfCSMAR = pd.concat(outputTogetherCSMAR)
togetherDfCSMAR = togetherDfCSMAR.drop_duplicates()
togetherDfCSMAR.columns = ['compcode','end_date','default_probability']
togetherDfCSMAR.to_excel('together_final_CSMAR.xlsx', index=False, encoding='utf_8_sig')
togetherWeeklyDfCSMAR = pd.concat(outputWeeklyTogetherCSMAR)
togetherWeeklyDfCSMAR = togetherWeeklyDfCSMAR.drop_duplicates()
togetherWeeklyDfCSMAR.columns = ['compcode','end_date','default_probability']
togetherWeeklyDfCSMAR.to_excel('together_weekly_return_final_CSMAR.xlsx', index=False, encoding='utf_8_sig')
os.chdir('../')

In [ ]:
# Summary explanation
'''
Tushare data results:
togetherDf for ln return
togetherWeeklyDf for weekly return

CSMAR data results:
togetherDfCSMAR for ln return
togetherWeeklyDfCSMAR for weekly return

'''

# Load in Default Data

In [ ]:
defaultData = pd.read_excel('default_SHSZ.xlsx')

In [ ]:
defaultCompcode = defaultData['tsCode'].tolist()
defaultCompdate = defaultData['compdate'].tolist()

newDef_compdate = []
for i in defaultCompdate:
    if type(i) == datetime.datetime:
        newDef_compdate.append(time.strftime("%Y%m%d",time.strptime(str(i), "%Y-%m-%d %H:%M:%S")))
    else:
        newDef_compdate.append(time.strftime("%Y%m%d",time.strptime(i, "%m/%d/%Y")))

In [ ]:
# Tushare data result merge
framesln = []
framesWeekly = []
for i in range(len(defaultCompcode)):
    df_ln = togetherDf[togetherDf['compcode'] == defaultCompcode[i]]
    df_ln['default_date'] = newDef_compdate[i]
    framesln.append(df_ln)
    df_weekly = togetherWeeklyDf[togetherWeeklyDf['compcode'] == defaultCompcode[i]]
    df_weekly['default_date'] = newDef_compdate[i]
    framesWeekly.append(df_weekly)
defaultExtractln = pd.concat(framesln)
defaultExtractln = defaultExtractln.drop_duplicates()
defaultExtractWeekly = pd.concat(framesWeekly)
defaultExtractWeekly = defaultExtractWeekly.drop_duplicates()
os.chdir('Default_Data/')
defaultExtractln.to_excel('default_prob_summary_ln.xlsx', index=False, encoding='utf_8_sig')
defaultExtractWeekly.to_excel('default_prob_summary_weekly.xlsx', index=False, encoding='utf_8_sig')
os.chdir('../')

In [ ]:
# Default Result with ln return, rf_last and Tushare debt data
defaultLnLastTuFrame = []
for i in list(set(defaultExtractln['compcode'].tolist())):
    result = defaultExtractln[defaultExtractln['compcode'] == i]
    if (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231') in result['end_date'].tolist():
        defaultLnLastTuFrame.append(result[result['end_date'] == (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231')])
    else:
        pass
defaultLnLastTu = pd.concat(defaultLnLastTuFrame)
os.chdir('Default_Data/final_result/')
defaultLnLastTu.to_excel('defaultLnLastTushare.xlsx', index=False, encoding='sig_utf_8')
fig = plt.figure(figsize=(25,10))
plt.bar(defaultLnLastTu['compcode'].tolist(), defaultLnLastTu['default_probability'].tolist())
fig.savefig('defaultLnLastTushare.jpg', dpi = 750)
plt.close()
os.chdir('../../')

In [ ]:
# Default Result with weekly return, rf_last and Tushare debt data
defaultWeeklyLastTuFrame = []
for i in list(set(defaultExtractWeekly['compcode'].tolist())):
    result = defaultExtractWeekly[defaultExtractWeekly['compcode'] == i]
    if (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231') in result['end_date'].tolist():
        defaultWeeklyLastTuFrame.append(result[result['end_date'] == (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231')])
    else:
        pass
defaultWeeklyLastTu = pd.concat(defaultWeeklyLastTuFrame)
os.chdir('Default_Data/final_result/')
defaultWeeklyLastTu.to_excel('defaultWeeklyLastTushare.xlsx', index=False, encoding='sig_utf_8')
fig = plt.figure(figsize=(25,10))
plt.bar(defaultWeeklyLastTu['compcode'].tolist(), defaultWeeklyLastTu['default_probability'].tolist())
fig.savefig('defaultWeeklyLastTushare.jpg', dpi = 750)
plt.close()
os.chdir('../../')

In [ ]:
# CSMAR data result merge
frameslnCSMAR = []
framesWeeklyCSMAR = []
for i in range(len(defaultCompcode)):
    df_ln_CSMAR = togetherDfCSMAR[togetherDfCSMAR['compcode'] == defaultCompcode[i]]
    df_ln_CSMAR['default_date'] = newDef_compdate[i]
    frameslnCSMAR.append(df_ln_CSMAR)
    df_weekly_CSMAR = togetherWeeklyDfCSMAR[togetherWeeklyDfCSMAR['compcode'] == defaultCompcode[i]]
    df_weekly_CSMAR['default_date'] = newDef_compdate[i]
    framesWeeklyCSMAR.append(df_weekly_CSMAR)
defaultExtractlnCSMAR = pd.concat(frameslnCSMAR)
defaultExtractlnCSMAR = defaultExtractlnCSMAR.drop_duplicates()
defaultExtractWeeklyCSMAR = pd.concat(framesWeeklyCSMAR)
defaultExtractWeeklyCSMAR = defaultExtractWeeklyCSMAR.drop_duplicates()
os.chdir('Default_Data/')
defaultExtractlnCSMAR.to_excel('default_prob_summary_ln_CSMAR.xlsx', index=False, encoding='utf_8_sig')
defaultExtractWeeklyCSMAR.to_excel('default_prob_summary_weekly_CSMAR.xlsx', index=False, encoding='utf_8_sig')
os.chdir('../')

In [ ]:
# Default Result with ln return, rf_last and CSMAR debt data
defaultLnLastCSMARFrame = []
for i in list(set(defaultExtractlnCSMAR['compcode'].tolist())):
    result = defaultExtractlnCSMAR[defaultExtractlnCSMAR['compcode'] == i]
    if (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231') in result['end_date'].tolist():
        defaultLnLastCSMARFrame.append(result[result['end_date'] == (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231')])
    else:
        pass
defaultLnLastCSMAR = pd.concat(defaultLnLastCSMARFrame)
os.chdir('Default_Data/final_result/')
defaultLnLastCSMAR.to_excel('defaultLnLastCSMAR.xlsx', index=False, encoding='sig_utf_8')
fig = plt.figure(figsize=(25,10))
plt.bar(defaultLnLastCSMAR['compcode'].tolist(), defaultLnLastCSMAR['default_probability'].tolist())
fig.savefig('defaultLnLastCSMAR.jpg', dpi = 750)
plt.close()
os.chdir('../../')

In [ ]:
# Default Result with weekly return, rf_last and CSMAR debt data
defaultWeeklyLastCSMARFrame = []
for i in list(set(defaultExtractWeeklyCSMAR['compcode'].tolist())):
    result = defaultExtractWeeklyCSMAR[defaultExtractWeeklyCSMAR['compcode'] == i]
    if (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231') in result['end_date'].tolist():
        defaultWeeklyLastCSMARFrame.append(result[result['end_date'] == (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231')])
    else:
        pass
defaultWeeklyLastCSMAR = pd.concat(defaultWeeklyLastCSMARFrame)
os.chdir('Default_Data/final_result/')
defaultWeeklyLastCSMAR.to_excel('defaultWeeklyLastCSMAR.xlsx', index=False, encoding='sig_utf_8')
fig = plt.figure(figsize=(25,10))
plt.bar(defaultWeeklyLastCSMAR['compcode'].tolist(), defaultWeeklyLastCSMAR['default_probability'].tolist())
fig.savefig('defaultWeeklyLastCSMAR.jpg', dpi = 750)
plt.close()
os.chdir('../../')

In [ ]:
os.getcwd()

# Use rf_mean as rf, and rerunning the whole things (Using)

In [ ]:
os.chdir('Default_Data/output_rf_mean/')

In [ ]:
# Data from Tushare
togetherFinal_rfmean = []
togetherWeeklyFinal_rfmean = []
for i in stockCode_SHSZ:
    print(i)
    # Debt
    debtData = pro.balancesheet(ts_code=i, start_date='20110101', end_date='20210630', fields='end_date,total_cur_liab,total_ncl,total_liab')
    debtNew=debtData[debtData['end_date'].apply(lambda x:x[4:])=='1231']
    debt=debtNew.drop_duplicates()
    shortTerm = debt['total_cur_liab'].tolist()
    longTerm = debt['total_ncl'].tolist()
    totalDebt = debt['total_liab'].tolist()
    endDate = debt['end_date'].tolist()
    
    # Calculate three kinds of DP
    dp = [] # DP
    dp_50 = [] # DP50
    dp_75 = [] # DP75
    d_facevalue = [] # D
    for y in year:
        for e in endDate:
            if e.startswith(y) == True:
                current = shortTerm[endDate.index(e)]
                long = longTerm[endDate.index(e)]
                if (current == None) or (long == None):
                    dp.append(nan)
                    dp_50.append(nan)
                    dp_75.append(nan)
                else:
                    dp.append(current)
                    dp_50.append(current+0.5*long)
                    dp_75.append(current+0.75*long)
                d_facevalue.append(totalDebt[endDate.index(e)])
            else:
                pass
    dpFrame = pd.DataFrame([lastDate,dp,dp_50,dp_75,d_facevalue]).T
    dpFrame.columns = ['end_date','DP','DP50','DP75','D']
    
    
    # BPS
    bpsData = pro.fina_indicator(ts_code=i, start_date='20110101', end_date='20210630', fields='ts_code,end_date,bps')
    d = dict(zip(bpsData['end_date'],bpsData['bps']))
    bpsData = []
    for j in d.keys():
        if j.endswith('1231') == True:
            bpsData.append([j,d[j]])
    bpsData = sorted(bpsData, key=lambda x:x[0], reverse=False)
    bps = pd.DataFrame(bpsData)
    bps.columns = ['end_date','bps']
    bpsData = bps['bps'].tolist()

    # Weekly Stock Return
    closeData = ts.pro_bar(ts_code=i, freq='W', adj='qfq', start_date='20110101', end_date='20210101')
    #closeData = pro.weekly(ts_code=i, start_date='20110101', end_date='20210101', fields='trade_date,close,pre_close')
    if type(closeData) == pd.DataFrame:
        closeData['ln_return'] = [math.log(closeData['close'][i]/closeData['pre_close'][i]) for i in range(len(closeData))]
        closeData['weekly_return'] = [(closeData['close'][i] - closeData['pre_close'][i])/closeData['pre_close'][i] for i in range(len(closeData))]
        closeData['square_ln'] = [pow(i,2) for i in closeData['ln_return'].tolist()]
        closeData['square_weekly'] = [pow(i,2) for i in closeData['weekly_return'].tolist()]
        closePrice = closeData['close'].tolist()
        lnList = closeData['ln_return'].tolist()
        weeklyList = closeData['weekly_return'].tolist()    
        squareLnList = closeData['square_ln'].tolist()
        squareWeeklyList = closeData['square_weekly'].tolist()
        tradeDate = closeData['trade_date'].tolist()
    else:
        pass
  
    
    # Start to calculate KMV
    sum_ln = []
    sum_weekly = []
    square_sum_ln = []
    square_sum_weekly = []
    mean_close = []
    nList = []
    for j in year:
        #print(j)
        sumln = []
        sumweekly = []
        squaresumln = []
        squaresumweekly = []
        meanClosePrice = []
        for k in tradeDate:
            if k.startswith(j) == True:
                sumln.append(lnList[tradeDate.index(k)])
                sumweekly.append(weeklyList[tradeDate.index(k)])
                squaresumln.append(squareLnList[tradeDate.index(k)])
                squaresumweekly.append(squareWeeklyList[tradeDate.index(k)])
                meanClosePrice.append(closePrice[tradeDate.index(k)])
            else:
                pass
        sum_ln.append(np.sum(sumln))
        sum_weekly.append(np.sum(sumweekly))        
        square_sum_ln.append(np.sum(squaresumln))
        square_sum_weekly.append(np.sum(squaresumweekly))
        if len(meanClosePrice) == 0:
            mean_close.append(0)
        else:
            mean_close.append(np.mean(meanClosePrice))
        nList.append(len(sumln))
    print('Done')
    weeklyReturn = pd.DataFrame([lastDate, sum_ln, sum_weekly, square_sum_ln, square_sum_weekly, mean_close, nList]).T
    weeklyReturn.columns = ['end_date','ln_return','weekly_return','square_ln_return','square_weekly_return','mean_close','numOfWeeks']
    
    # Calculate E
    # First, get the number of tradable shares and non-tradable shares
    shareData = capchgData.loc[capchgData['Stkcd'] == i]
    tradeShare = []
    nonTradeShare = []
    for l in lastDate:
        if l <= shareData['Shrchgdt'].tolist()[0]:
            total = 0
            trade = 0
        else:
            total = shareData.loc[shareData['Shrchgdt'] <= l]['Nshrttl'].tolist()[-1]
            trade = shareData.loc[shareData['Shrchgdt'] <= l]['Nshra'].tolist()[-1]
        nonTrade = total - trade
        tradeShare.append(trade)
        nonTradeShare.append(nonTrade)
    # Then calculate E
    equity = [] # E
    for r in range(len(bps)):
        tradeEquity = mean_close[r] * tradeShare[r]
        nonTradeEquity = bpsData[r] * nonTradeShare[r]
        equityMktVal = tradeEquity + nonTradeEquity
        equity.append(equityMktVal)
        
    # Use ln_Return to calculate sigmaE
    sigma_E_ln = []
    for ye in range(len(year)):
        sigma_E_ln.append(sigmaE(nList[ye], square_sum_ln[ye], sum_ln[ye]))
    eAndSigmaELn = pd.DataFrame([lastDate,equity,sigma_E_ln,rf_mean]).T
    eAndSigmaELn.columns = ['end_date','E','sigmaE','r']
    
    totalDf = dpFrame.merge(debt, on='end_date',how='left')
    totalDf = totalDf.merge(bps, on='end_date',how='left')
    totalDf = totalDf.merge(weeklyReturn, on='end_date',how='left')
    totalDf = totalDf.merge(eAndSigmaELn, on='end_date',how='left')
    finalDf = totalDf.dropna(axis=0,how='any')
    

    output = []
    longth = finalDf.shape[0]
    compcode = []
    for ye in range(longth):
        compcode.append(i)
        Equity = finalDf['E'].tolist()[ye]
        SigmaE = finalDf['sigmaE'].tolist()[ye]
        D = finalDf['D'].tolist()[ye]
        r = finalDf['r'].tolist()[ye]/100
        medium =f(Equity,SigmaE,D,r,1)
        if medium[0] == 0 or medium[1] == 0:
            output.append('-')
        else:
            dd = (medium[0] - finalDf['DP50'].tolist()[ye])/(medium[0]*medium[1])
            output.append(1-N(dd))
    outputDf = pd.DataFrame([finalDf['end_date'].tolist(),output]).T
    outputDf.columns = ['end_date','default_probability']
    outputDf.to_excel(i+'.xlsx', index=False, encoding='utf_8_sig')
    
    if len(compcode) == 0:
        pass
    else:
        togetherOutput = pd.DataFrame([compcode, finalDf['end_date'].tolist(), output]).T
        togetherFinal_rfmean.append(togetherOutput)
    
    
    

    # Use Weekly_Return to calculate sigmaE
    sigma_E_weekly = []
    for ye in range(len(year)):
        sigma_E_weekly.append(sigmaE(nList[ye], square_sum_weekly[ye], sum_weekly[ye]))
    eAndSigmaEweekly = pd.DataFrame([lastDate,equity,sigma_E_weekly,rf_mean]).T
    eAndSigmaEweekly.columns = ['end_date','E','sigmaE','r']
    
    totalDfWeekly = dpFrame.merge(debt, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(bps, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(weeklyReturn, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(eAndSigmaEweekly, on='end_date',how='left')
    finalDfWeekly = totalDfWeekly.dropna(axis=0,how='any')
    

    outputWeekly = []
    longthWeekly = finalDfWeekly.shape[0]
    compcodeWeekly = []
    for ye in range(longthWeekly):
        compcodeWeekly.append(i)
        EquityWeekly = finalDfWeekly['E'].tolist()[ye]
        SigmaEWeekly = finalDfWeekly['sigmaE'].tolist()[ye]
        DWeekly = finalDfWeekly['D'].tolist()[ye]
        rWeekly = finalDfWeekly['r'].tolist()[ye]/100
        mediumWeekly = f(EquityWeekly,SigmaEWeekly,DWeekly,rWeekly,1)
        if mediumWeekly[0] == 0 or mediumWeekly[1] == 0:
            outputWeekly.append('-')
        else:
            dd = (mediumWeekly[0] - finalDfWeekly['DP50'].tolist()[ye])/(mediumWeekly[0]*mediumWeekly[1])
            outputWeekly.append(1-N(dd))
    outputDfWeekly = pd.DataFrame([finalDfWeekly['end_date'].tolist(),outputWeekly]).T
    outputDfWeekly.columns = ['end_date','default_probability']
    outputDfWeekly.to_excel(i+'_weekly.xlsx', index=False, encoding='utf_8_sig')
    
    if len(compcodeWeekly) == 0:
        pass
    else:
        togetherWeeklyOutput = pd.DataFrame([compcodeWeekly, finalDfWeekly['end_date'].tolist(), outputWeekly]).T
        togetherWeeklyFinal_rfmean.append(togetherWeeklyOutput)
    
    

In [ ]:
os.chdir('../')
outputTogether_rfmean = togetherFinal_rfmean
outputWeeklyTogether_rfmean = togetherWeeklyFinal_rfmean
togetherDf_rfmean = pd.concat(outputTogether_rfmean)
togetherDf_rfmean = togetherDf_rfmean.drop_duplicates()
togetherDf_rfmean.columns = ['compcode','end_date','default_probability']
togetherDf_rfmean.to_excel('together_final_rfmean.xlsx', index=False, encoding='utf_8_sig')
togetherWeeklyDf_rfmean = pd.concat(outputWeeklyTogether_rfmean)
togetherWeeklyDf_rfmean = togetherWeeklyDf_rfmean.drop_duplicates()
togetherWeeklyDf_rfmean.columns = ['compcode','end_date','default_probability']
togetherWeeklyDf_rfmean.to_excel('together_weekly_return_final_rfmean.xlsx', index=False, encoding='utf_8_sig')
os.chdir('../')

In [ ]:
os.chdir('Default_Data/output_rf_mean_CSMAR/')

In [ ]:
# Data from CSMAR
togetherFinalCSMAR_rfmean = []
togetherWeeklyFinalCSMAR_rfmean = []
for i in stockCode_SHSZ:
    print(i)
    # Debt
    debtUse = combasData[combasData['Stkcd'] == i[:6]]
    shortTerm = debtUse['current_debt'].tolist()
    longTerm = debtUse['long_term_debt'].tolist()
    totalDebt = debtUse['total_debt'].tolist()
    endDate = [x.replace('-','') for x in debtUse['end_date'].tolist()]

    
    # Calculate three kinds of DP
    dp = [] # DP
    dp_50 = [] # DP50
    dp_75 = [] # DP75
    d_facevalue = [] # D
    for y in year:
        for e in endDate:
            if e.startswith(y) == True:
                current = shortTerm[endDate.index(e)]
                long = longTerm[endDate.index(e)]
                if (current == None) or (long == None):
                    dp.append(nan)
                    dp_50.append(nan)
                    dp_75.append(nan)
                else:
                    dp.append(current)
                    dp_50.append(current+0.5*long)
                    dp_75.append(current+0.75*long)
                d_facevalue.append(totalDebt[endDate.index(e)])
            else:
                pass
    dpFrame = pd.DataFrame([lastDate,dp,dp_50,dp_75,d_facevalue]).T
    dpFrame.columns = ['end_date','DP','DP50','DP75','D']
    
    
    # BPS
    bpsData = pro.fina_indicator(ts_code=i, start_date='20110101', end_date='20210630', fields='ts_code,end_date,bps')
    d = dict(zip(bpsData['end_date'],bpsData['bps']))
    bpsData = []
    for j in d.keys():
        if j.endswith('1231') == True:
            bpsData.append([j,d[j]])
    bpsData = sorted(bpsData, key=lambda x:x[0], reverse=False)
    bps = pd.DataFrame(bpsData)
    bps.columns = ['end_date','bps']
    bpsData = bps['bps'].tolist()

    # Weekly Stock Return
    closeData = ts.pro_bar(ts_code=i, freq='W', adj='qfq', start_date='20110101', end_date='20210101')
    #closeData = pro.weekly(ts_code=i, start_date='20110101', end_date='20210101', fields='trade_date,close,pre_close')
    if type(closeData) == pd.DataFrame:
        closeData['ln_return'] = [math.log(closeData['close'][i]/closeData['pre_close'][i]) for i in range(len(closeData))]
        closeData['weekly_return'] = [(closeData['close'][i] - closeData['pre_close'][i])/closeData['pre_close'][i] for i in range(len(closeData))]
        closeData['square_ln'] = [pow(i,2) for i in closeData['ln_return'].tolist()]
        closeData['square_weekly'] = [pow(i,2) for i in closeData['weekly_return'].tolist()]
        closePrice = closeData['close'].tolist()
        lnList = closeData['ln_return'].tolist()
        weeklyList = closeData['weekly_return'].tolist()    
        squareLnList = closeData['square_ln'].tolist()
        squareWeeklyList = closeData['square_weekly'].tolist()
        tradeDate = closeData['trade_date'].tolist()
    else:
        pass
  
    
    # Start to calculate KMV
    sum_ln = []
    sum_weekly = []
    square_sum_ln = []
    square_sum_weekly = []
    mean_close = []
    nList = []
    for j in year:
        #print(j)
        sumln = []
        sumweekly = []
        squaresumln = []
        squaresumweekly = []
        meanClosePrice = []
        for k in tradeDate:
            if k.startswith(j) == True:
                sumln.append(lnList[tradeDate.index(k)])
                sumweekly.append(weeklyList[tradeDate.index(k)])
                squaresumln.append(squareLnList[tradeDate.index(k)])
                squaresumweekly.append(squareWeeklyList[tradeDate.index(k)])
                meanClosePrice.append(closePrice[tradeDate.index(k)])
            else:
                pass
        sum_ln.append(np.sum(sumln))
        sum_weekly.append(np.sum(sumweekly))        
        square_sum_ln.append(np.sum(squaresumln))
        square_sum_weekly.append(np.sum(squaresumweekly))
        if len(meanClosePrice) == 0:
            mean_close.append(0)
        else:
            mean_close.append(np.mean(meanClosePrice))
        nList.append(len(sumln))
    print('Done')
    weeklyReturn = pd.DataFrame([lastDate, sum_ln, sum_weekly, square_sum_ln, square_sum_weekly, mean_close, nList]).T
    weeklyReturn.columns = ['end_date','ln_return','weekly_return','square_ln_return','square_weekly_return','mean_close','numOfWeeks']
    
    # Calculate E
    # First, get the number of tradable shares and non-tradable shares
    shareData = capchgData.loc[capchgData['Stkcd'] == i]
    tradeShare = []
    nonTradeShare = []
    for l in lastDate:
        if l <= shareData['Shrchgdt'].tolist()[0]:
            total = 0
            trade = 0
        else:
            total = shareData.loc[shareData['Shrchgdt'] <= l]['Nshrttl'].tolist()[-1]
            trade = shareData.loc[shareData['Shrchgdt'] <= l]['Nshra'].tolist()[-1]
        nonTrade = total - trade
        tradeShare.append(trade)
        nonTradeShare.append(nonTrade)
    # Then calculate E
    equity = [] # E
    for r in range(len(bps)):
        tradeEquity = mean_close[r] * tradeShare[r]
        nonTradeEquity = bpsData[r] * nonTradeShare[r]
        equityMktVal = tradeEquity + nonTradeEquity
        equity.append(equityMktVal)
        
    # Use ln_Return to calculate sigmaE
    sigma_E_ln = []
    for ye in range(len(year)):
        sigma_E_ln.append(sigmaE(nList[ye], square_sum_ln[ye], sum_ln[ye]))
    eAndSigmaELn = pd.DataFrame([lastDate,equity,sigma_E_ln,rf_mean]).T
    eAndSigmaELn.columns = ['end_date','E','sigmaE','r']
    
    totalDf = dpFrame.merge(debt, on='end_date',how='left')
    totalDf = totalDf.merge(bps, on='end_date',how='left')
    totalDf = totalDf.merge(weeklyReturn, on='end_date',how='left')
    totalDf = totalDf.merge(eAndSigmaELn, on='end_date',how='left')
    finalDf = totalDf.dropna(axis=0,how='any')
    

    output = []
    longth = finalDf.shape[0]
    compcode = []
    for ye in range(longth):
        compcode.append(i)
        Equity = finalDf['E'].tolist()[ye]
        SigmaE = finalDf['sigmaE'].tolist()[ye]
        D = finalDf['D'].tolist()[ye]
        r = finalDf['r'].tolist()[ye]/100
        medium =f(Equity,SigmaE,D,r,1)
        if medium[0] == 0 or medium[1] == 0:
            output.append('-')
        else:
            dd = (medium[0] - finalDf['DP50'].tolist()[ye])/(medium[0]*medium[1])
            output.append(1-N(dd))
    outputDf = pd.DataFrame([finalDf['end_date'].tolist(),output]).T
    outputDf.columns = ['end_date','default_probability']
    outputDf.to_excel(i+'.xlsx', index=False, encoding='utf_8_sig')
    
    if len(compcode) == 0:
        pass
    else:
        togetherOutput = pd.DataFrame([compcode, finalDf['end_date'].tolist(), output]).T
        togetherFinalCSMAR_rfmean.append(togetherOutput)

    # Use Weekly_Return to calculate sigmaE
    sigma_E_weekly = []
    for ye in range(len(year)):
        sigma_E_weekly.append(sigmaE(nList[ye], square_sum_weekly[ye], sum_weekly[ye]))
    eAndSigmaEweekly = pd.DataFrame([lastDate,equity,sigma_E_weekly,rf_mean]).T
    eAndSigmaEweekly.columns = ['end_date','E','sigmaE','r']
    
    totalDfWeekly = dpFrame.merge(debt, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(bps, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(weeklyReturn, on='end_date',how='left')
    totalDfWeekly = totalDfWeekly.merge(eAndSigmaEweekly, on='end_date',how='left')
    finalDfWeekly = totalDfWeekly.dropna(axis=0,how='any')
    

    outputWeekly = []
    longthWeekly = finalDfWeekly.shape[0]
    compcodeWeekly = []
    for ye in range(longthWeekly):
        compcodeWeekly.append(i)
        EquityWeekly = finalDfWeekly['E'].tolist()[ye]
        SigmaEWeekly = finalDfWeekly['sigmaE'].tolist()[ye]
        DWeekly = finalDfWeekly['D'].tolist()[ye]
        rWeekly = finalDfWeekly['r'].tolist()[ye]/100
        mediumWeekly = f(EquityWeekly,SigmaEWeekly,DWeekly,rWeekly,1)
        if mediumWeekly[0] == 0 or mediumWeekly[1] == 0:
            outputWeekly.append('-')
        else:
            dd = (mediumWeekly[0] - finalDfWeekly['DP50'].tolist()[ye])/(mediumWeekly[0]*mediumWeekly[1])
            outputWeekly.append(1-N(dd))
    outputDfWeekly = pd.DataFrame([finalDfWeekly['end_date'].tolist(),outputWeekly]).T
    outputDfWeekly.columns = ['end_date','default_probability']
    outputDfWeekly.to_excel(i+'_weekly.xlsx', index=False, encoding='utf_8_sig')
    
    if len(compcodeWeekly) == 0:
        pass
    else:
        togetherWeeklyOutput = pd.DataFrame([compcodeWeekly, finalDfWeekly['end_date'].tolist(), outputWeekly]).T
        togetherWeeklyFinalCSMAR_rfmean.append(togetherWeeklyOutput)
    
    

In [ ]:
os.chdir('../')
outputTogetherCSMAR_rfmean = togetherFinalCSMAR_rfmean
outputWeeklyTogetherCSMAR_rfmean = togetherWeeklyFinalCSMAR_rfmean
togetherDfCSMAR_rfmean = pd.concat(outputTogetherCSMAR_rfmean)
togetherDfCSMAR_rfmean = togetherDfCSMAR_rfmean.drop_duplicates()
togetherDfCSMAR_rfmean.columns = ['compcode','end_date','default_probability']
togetherDfCSMAR_rfmean.to_excel('together_final_CSMAR_rfmean.xlsx', index=False, encoding='utf_8_sig')
togetherWeeklyDfCSMAR_rfmean = pd.concat(outputWeeklyTogetherCSMAR_rfmean)
togetherWeeklyDfCSMAR_rfmean = togetherWeeklyDfCSMAR_rfmean.drop_duplicates()
togetherWeeklyDfCSMAR_rfmean.columns = ['compcode','end_date','default_probability']
togetherWeeklyDfCSMAR_rfmean.to_excel('together_weekly_return_final_CSMAR_rfmean.xlsx', index=False, encoding='utf_8_sig')
os.chdir('../')

In [ ]:
# Tushare data result merge
framesln_rfmean = []
framesWeekly_rfmean = []
for i in range(len(defaultCompcode)):
    df_ln_rfmean = togetherDf_rfmean[togetherDf_rfmean['compcode'] == defaultCompcode[i]]
    df_ln_rfmean['default_date'] = newDef_compdate[i]
    framesln_rfmean.append(df_ln_rfmean)
    df_weekly_rfmean = togetherWeeklyDf_rfmean[togetherWeeklyDf_rfmean['compcode'] == defaultCompcode[i]]
    df_weekly_rfmean['default_date'] = newDef_compdate[i]
    framesWeekly_rfmean.append(df_weekly_rfmean)
defaultExtractln_rfmean = pd.concat(framesln_rfmean)
defaultExtractln_rfmean = defaultExtractln_rfmean.drop_duplicates()
defaultExtractWeekly_rfmean = pd.concat(framesWeekly_rfmean)
defaultExtractWeekly_rfmean = defaultExtractWeekly_rfmean.drop_duplicates()
os.chdir('Default_Data/')
defaultExtractln_rfmean.to_excel('default_prob_summary_ln_rfmean.xlsx', index=False, encoding='utf_8_sig')
defaultExtractWeekly_rfmean.to_excel('default_prob_summary_weekly_rfmean.xlsx', index=False, encoding='utf_8_sig')
os.chdir('../')

In [ ]:
# Default Result with ln return, rf_mean and Tushare debt data
defaultLnMeanTuFrame = []
for i in list(set(defaultExtractln_rfmean['compcode'].tolist())):
    result = defaultExtractln_rfmean[defaultExtractln_rfmean['compcode'] == i]
    if (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231') in result['end_date'].tolist():
        defaultLnMeanTuFrame.append(result[result['end_date'] == (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231')])
    else:
        pass
defaultLnMeanTu = pd.concat(defaultLnMeanTuFrame)
os.chdir('Default_Data/final_result/')
defaultLnMeanTu.to_excel('defaultLnMeanTushare.xlsx', index=False, encoding='sig_utf_8')
fig = plt.figure(figsize=(25,10))
plt.bar(defaultLnMeanTu['compcode'].tolist(), defaultLnMeanTu['default_probability'].tolist())
fig.savefig('defaultLnMeanTushare.jpg', dpi = 750)
plt.close()
os.chdir('../../')

In [ ]:
# Default Result with weekly return, rf_mean and Tushare debt data
defaultWeeklyMeanTuFrame = []
for i in list(set(defaultExtractWeekly_rfmean['compcode'].tolist())):
    result = defaultExtractWeekly_rfmean[defaultExtractWeekly_rfmean['compcode'] == i]
    if (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231') in result['end_date'].tolist():
        defaultWeeklyMeanTuFrame.append(result[result['end_date'] == (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231')])
    else:
        pass
defaultWeeklyMeanTu = pd.concat(defaultWeeklyMeanTuFrame)
os.chdir('Default_Data/final_result/')
defaultWeeklyMeanTu.to_excel('defaultWeeklyMeanTushare.xlsx', index=False, encoding='sig_utf_8')
fig = plt.figure(figsize=(25,10))
plt.bar(defaultWeeklyMeanTu['compcode'].tolist(), defaultWeeklyMeanTu['default_probability'].tolist())
fig.savefig('defaultWeeklyMeanTushare.jpg', dpi = 750)
plt.close()
os.chdir('../../')

In [ ]:
# CSMAR data result merge
frameslnCSMAR_rfmean = []
framesWeeklyCSMAR_rfmean = []
for i in range(len(defaultCompcode)):
    df_ln_CSMAR_rfmean = togetherDfCSMAR_rfmean[togetherDfCSMAR_rfmean['compcode'] == defaultCompcode[i]]
    df_ln_CSMAR_rfmean['default_date'] = newDef_compdate[i]
    frameslnCSMAR_rfmean.append(df_ln_CSMAR_rfmean)
    df_weekly_CSMAR_rfmean = togetherWeeklyDfCSMAR_rfmean[togetherWeeklyDfCSMAR_rfmean['compcode'] == defaultCompcode[i]]
    df_weekly_CSMAR_rfmean['default_date'] = newDef_compdate[i]
    framesWeeklyCSMAR_rfmean.append(df_weekly_CSMAR_rfmean)
defaultExtractlnCSMAR_rfmean = pd.concat(frameslnCSMAR_rfmean)
defaultExtractlnCSMAR_rfmean = defaultExtractlnCSMAR_rfmean.drop_duplicates()
defaultExtractWeeklyCSMAR_rfmean = pd.concat(framesWeeklyCSMAR_rfmean)
defaultExtractWeeklyCSMAR_rfmean = defaultExtractWeeklyCSMAR_rfmean.drop_duplicates()
os.chdir('Default_Data/')
defaultExtractlnCSMAR_rfmean.to_excel('default_prob_summary_ln_CSMAR_rfmean.xlsx', index=False, encoding='utf_8_sig')
defaultExtractWeeklyCSMAR_rfmean.to_excel('default_prob_summary_weekly_CSMAR_rfmean.xlsx', index=False, encoding='utf_8_sig')
os.chdir('../')

In [ ]:
# Default Result with ln return, rf_mean and CSMAR debt data
defaultLnMeanCSMARFrame = []
for i in list(set(defaultExtractlnCSMAR_rfmean['compcode'].tolist())):
    result = defaultExtractlnCSMAR_rfmean[defaultExtractlnCSMAR_rfmean['compcode'] == i]
    if (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231') in result['end_date'].tolist():
        defaultLnMeanCSMARFrame.append(result[result['end_date'] == (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231')])
    else:
        pass
defaultLnMeanCSMAR = pd.concat(defaultLnMeanCSMARFrame)
os.chdir('Default_Data/final_result/')
defaultLnMeanCSMAR.to_excel('defaultLnMeanCSMAR.xlsx', index=False, encoding='sig_utf_8')
fig = plt.figure(figsize=(25,10))
plt.bar(defaultLnMeanCSMAR['compcode'].tolist(), defaultLnMeanCSMAR['default_probability'].tolist())
fig.savefig('defaultLnMeanCSMAR.jpg', dpi = 750)
plt.close()
os.chdir('../../')

In [ ]:
# Default Result with weekly return, rf_mean and CSMAR debt data
defaultWeeklyMeanCSMARFrame = []
for i in list(set(defaultExtractWeeklyCSMAR_rfmean['compcode'].tolist())):
    result = defaultExtractWeeklyCSMAR_rfmean[defaultExtractWeeklyCSMAR_rfmean['compcode'] == i]
    if (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231') in result['end_date'].tolist():
        defaultWeeklyMeanCSMARFrame.append(result[result['end_date'] == (str(eval(result['default_date'].tolist()[0][:4]) - 1) + '1231')])
    else:
        pass
defaultWeeklyMeanCSMAR = pd.concat(defaultWeeklyMeanCSMARFrame)
os.chdir('Default_Data/final_result/')
defaultWeeklyMeanCSMAR.to_excel('defaultWeeklyMeanCSMAR.xlsx', index=False, encoding='sig_utf_8')
fig = plt.figure(figsize=(25,10))
plt.bar(defaultWeeklyMeanCSMAR['compcode'].tolist(), defaultWeeklyMeanCSMAR['default_probability'].tolist())
fig.savefig('defaultWeeklyMeanCSMAR.jpg', dpi = 750)
plt.close()
os.chdir('../../')

# Summary the Tushare and CSMAR, rf_mean, ln_return result to prepare for the plot

In [ ]:
huizong = []
for i in stockCode_SHSZ:
    togetherall = pd.DataFrame(lastDate)
    togetherall.columns = ['end_date']
    togetherall['compcode'] = i
    huizong.append(togetherall)
resultDataframe = pd.concat(huizong)
resultDataframeTushare = resultDataframe
resultDataframeCSMAR = resultDataframe

In [ ]:
# Tushare merge
tushareMerge = resultDataframeTushare.merge(togetherDf_rfmean, on=['end_date','compcode'], how='left')
defaultLastDate = []
for i in newDef_compdate:
    defaultLastDate.append(str(eval(i[:4])-1)+'1231')

In [ ]:
default_Date = []
for i in tushareMerge['compcode'].tolist():
    if i in defaultCompcode:
        default_Date.append(defaultLastDate[defaultCompcode.index(i)])
    else:
        default_Date.append('0')
tushareMerge['default_date'] = pd.DataFrame(default_Date)

In [ ]:
delete_line = []
for j in range(tushareMerge.shape[0]):
    if (tushareMerge['compcode'].tolist()[j] in defaultCompcode) and (tushareMerge['default_date'].tolist()[j] < tushareMerge['end_date'].tolist()[j]):
        delete_line.append(j)
    else:
        pass
tushareMergeFinal = tushareMerge.drop(labels=delete_line, axis=0)

In [ ]:
tushareMergeResult = tushareMergeFinal.reset_index().drop(['index'], axis=1)

In [ ]:
defaultJudge = []
for i in range(tushareMergeResult.shape[0]):
    if tushareMergeResult['default_date'].tolist()[i] == tushareMergeResult['end_date'].tolist()[i]:
        defaultJudge.append(1)
    else:
        defaultJudge.append(0)
tushareMergeResult['default_judgment'] = pd.DataFrame(defaultJudge)
tushareMergeResult = tushareMergeResult.drop(['default_date'],axis=1)
tushareMergeResult.to_excel('tushareMergeFinal.xlsx', index=False, encoding='sig_utf_8')

In [ ]:
# CSMAR merge
CSMARMerge = resultDataframeCSMAR.merge(togetherDfCSMAR_rfmean, on=['end_date','compcode'], how='left')
defaultLastDate = []
for i in newDef_compdate:
    defaultLastDate.append(str(eval(i[:4])-1)+'1231')
default_Date = []
for i in CSMARMerge['compcode'].tolist():
    if i in defaultCompcode:
        default_Date.append(defaultLastDate[defaultCompcode.index(i)])
    else:
        default_Date.append('0')
CSMARMerge['default_date'] = pd.DataFrame(default_Date)
delete_line = []
for j in range(CSMARMerge.shape[0]):
    if (CSMARMerge['compcode'].tolist()[j] in defaultCompcode) and (CSMARMerge['default_date'].tolist()[j] < CSMARMerge['end_date'].tolist()[j]):
        delete_line.append(j)
    else:
        pass
CSMARMergeFinal = CSMARMerge.drop(labels=delete_line, axis=0)
CSMARMergeResult = CSMARMergeFinal.reset_index().drop(['index'], axis=1)
defaultJudge = []
for i in range(CSMARMergeResult.shape[0]):
    if CSMARMergeResult['default_date'].tolist()[i] == CSMARMergeResult['end_date'].tolist()[i]:
        defaultJudge.append(1)
    else:
        defaultJudge.append(0)
CSMARMergeResult['default_judgment'] = pd.DataFrame(defaultJudge)
CSMARMergeResult = CSMARMergeResult.drop(['default_date'],axis=1)
CSMARMergeResult.to_excel('CSMARMergeFinal.xlsx', index=False, encoding='sig_utf_8')

# Plot the predictions

In [ ]:
# First input the preparation Tushare data file
plotData = tushareMergeResult
plotUseData = plotData[['default_probability','default_judgment']]
defaultPart = plotUseData[plotUseData['default_judgment'] == 1]
notDefaultPart = plotUseData[plotUseData['default_judgment'] == 0]
notDefaultPart = notDefaultPart[notDefaultPart['default_probability'] != '-']
defaultX = defaultPart['default_probability'].tolist()
defaultY = defaultPart['default_judgment'].tolist()
notDefaultX = notDefaultPart['default_probability'].tolist()
notDefaultY = notDefaultPart['default_judgment'].tolist()
# Start to plot with Tushare Data
fig = plt.figure(figsize=(25,10))
plt.scatter(defaultX, defaultY, s=16., color=(0.,0.5,0.))
plt.scatter(notDefaultX, notDefaultY, s=16., color=(0.5,0.,0.5))
plt.show()
#fig.savefig('tusharePlotData.jpg', dpi = 750)

In [ ]:
# Then input the preparation CSMAR data file
plotData = CSMARMergeResult
plotUseData = plotData[['default_probability','default_judgment']]
defaultPart = plotUseData[plotUseData['default_judgment'] == 1]
notDefaultPart = plotUseData[plotUseData['default_judgment'] == 0]
notDefaultPart = notDefaultPart[notDefaultPart['default_probability'] != '-']
defaultX = defaultPart['default_probability'].tolist()
defaultY = defaultPart['default_judgment'].tolist()
notDefaultX = notDefaultPart['default_probability'].tolist()
notDefaultY = notDefaultPart['default_judgment'].tolist()
# Start to plot with Tushare Data
fig = plt.figure(figsize=(25,10))
plt.scatter(defaultX, defaultY, s=16., color=(0.,0.5,0.))
plt.scatter(notDefaultX, notDefaultY, s=16., color=(0.5,0.,0.5))
plt.show()

In [ ]:
# First input the preparation Tushare data file
plotData = tushareMergeResult
plotUseData = plotData[['default_probability','default_judgment']]
defaultPart = plotUseData[plotUseData['default_judgment'] == 1]
notDefaultPart = plotUseData[plotUseData['default_judgment'] == 0]
notDefaultPart = notDefaultPart[notDefaultPart['default_probability'] != '-']
defaultX = defaultPart['default_probability'].tolist()
defaultY = defaultPart['default_judgment'].tolist()
notDefaultX = notDefaultPart['default_probability'].tolist()
notDefaultY = notDefaultPart['default_judgment'].tolist()
# Start to plot with Tushare Data
fig = plt.figure(figsize=(25,20))
plt.subplot(211)
plt.hist(defaultX, bins=50, color='steelblue')
plt.subplot(212)
plt.hist(notDefaultX, bins=50, color='red')
plt.show()
os.chdir('Default_Data/Plot/')
fig.savefig('tushareHistogram.jpg', dpi = 750)
os.chdir('../../')

In [ ]:
# Then input the preparation CSMAR data file
plotData = CSMARMergeResult
plotUseData = plotData[['default_probability','default_judgment']]
defaultPart = plotUseData[plotUseData['default_judgment'] == 1]
notDefaultPart = plotUseData[plotUseData['default_judgment'] == 0]
notDefaultPart = notDefaultPart[notDefaultPart['default_probability'] != '-']
defaultX = defaultPart['default_probability'].tolist()
defaultY = defaultPart['default_judgment'].tolist()
notDefaultX = notDefaultPart['default_probability'].tolist()
notDefaultY = notDefaultPart['default_judgment'].tolist()
# Start to plot with Tushare Data
fig = plt.figure(figsize=(25,20))
plt.subplot(211)
plt.hist(defaultX, bins=50, color='steelblue')
plt.subplot(212)
plt.hist(notDefaultX, bins=50, color='red')
plt.show()
os.chdir('Default_Data/Plot/')
fig.savefig('CSMARHistogram.jpg', dpi = 750)
os.chdir('../../')

In [ ]:
os.chdir('../../')

In [ ]:
os.getcwd()

In [ ]:
# First input the preparation Tushare data file
os.chdir('Default_Data/Plot/')
plotData = tushareMergeResult
plotUseData = plotData[['default_probability','default_judgment']]
defaultPart = plotUseData[plotUseData['default_judgment'] == 1]
notDefaultPart = plotUseData[plotUseData['default_judgment'] == 0]
notDefaultPart = notDefaultPart[notDefaultPart['default_probability'] != '-']
defaultX = defaultPart['default_probability'].tolist()
defaultY = defaultPart['default_judgment'].tolist()
notDefaultX = notDefaultPart['default_probability'].tolist()
notDefaultY = notDefaultPart['default_judgment'].tolist()

totalDefaultX = defaultX + notDefaultX
plotX = [0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]

count_005 = 0
count_015 = 0
count_025 = 0
count_035 = 0
count_045 = 0
count_055 = 0
count_065 = 0
count_075 = 0
count_085 = 0
count_095 = 0
for i in totalDefaultX:
    if 0 <= i < 0.1:
        count_005 += 1
    elif 0.1 <= i < 0.2:
        count_015 +=1
    elif 0.2 <= i < 0.3:
        count_025 +=1
    elif 0.3 <= i < 0.4:
        count_035 +=1
    elif 0.4 <= i < 0.5:
        count_045 +=1
    elif 0.5 <= i < 0.6:
        count_055 +=1
    elif 0.6 <= i < 0.7:
        count_065 +=1
    elif 0.7 <= i < 0.8:
        count_075 +=1
    elif 0.8 <= i <0.9:
        count_085 +=1
    elif 0.9 <= i <= 1.0:
        count_095 +=1
plotY = [count_005,count_015,count_025,count_035,count_045,count_055,count_065,count_075,count_085,count_095]

count_default_005 = 0
count_default_015 = 0
count_default_025 = 0
count_default_035 = 0
count_default_045 = 0
count_default_055 = 0
count_default_065 = 0
count_default_075 = 0
count_default_085 = 0
count_default_095 = 0
for i in defaultX:
    if 0 <= i < 0.1:
        count_default_005 += 1
    elif 0.1 <= i < 0.2:
        count_default_015 +=1
    elif 0.2 <= i < 0.3:
        count_default_025 +=1
    elif 0.3 <= i < 0.4:
        count_default_035 +=1
    elif 0.4 <= i < 0.5:
        count_default_045 +=1
    elif 0.5 <= i < 0.6:
        count_default_055 +=1
    elif 0.6 <= i < 0.7:
        count_default_065 +=1
    elif 0.7 <= i < 0.8:
        count_default_075 +=1
    elif 0.8 <= i <0.9:
        count_default_085 +=1
    elif 0.9 <= i <= 1.0:
        count_default_095 +=1
plotDefaultY = [100*count_default_005/count_005,100*count_default_015/count_015,100*count_default_025/count_025,
                100*count_default_035/count_035,100*count_default_045/count_045,100*count_default_055/count_055,
                100*count_default_065/count_065,100*count_default_075/count_075,100*count_default_085/count_085,
                100*count_default_095/count_095]


# Start to plot with Tushare Data
fig = plt.figure(figsize=(40,30))

ax1 = fig.add_subplot(111)


cmap_names = ["rainbow"]
for i, name in enumerate(cmap_names):
    cmap = mpl.cm.get_cmap(name, len(plotX))
    colors = cmap(np.linspace(0,1,len(plotX)))
    ax1.bar(plotX, plotY, width=0.05, color=colors, edgecolor='black')

ax1.set_ylabel('Number of Issuers',size=50)
ax1.set_xlabel('Default probability predictions',size=50)
plt.yticks(fontsize=50)
plt.xticks(np.arange(0,1,0.1), fontsize=50)

ax2 = ax1.twinx()
ax2.plot(plotX, plotDefaultY, color='red', label='Default rate')
ax2.set_ylabel('Actual default rate (%)',size=50)
plt.yticks(np.arange(0,6,1),fontsize=50)
plt.title('KMV - Bond Issuer Default Rate Prediction',fontsize=50)
plt.legend(loc='best',fontsize=40)


plt.show()
fig.savefig('tushareBar.jpg', dpi = 750)
os.chdir('../../')


In [ ]:
cmap_names = ["rainbow"]
for i, name in enumerate(cmap_names):
    print(i, name)

In [ ]:
# Then input the preparation CSMAR data file
os.chdir('Default_Data/Plot/')
plotData = CSMARMergeResult
plotUseData = plotData[['default_probability','default_judgment']]
defaultPart = plotUseData[plotUseData['default_judgment'] == 1]
notDefaultPart = plotUseData[plotUseData['default_judgment'] == 0]
notDefaultPart = notDefaultPart[notDefaultPart['default_probability'] != '-']
defaultX = defaultPart['default_probability'].tolist()
defaultY = defaultPart['default_judgment'].tolist()
notDefaultX = notDefaultPart['default_probability'].tolist()
notDefaultY = notDefaultPart['default_judgment'].tolist()


totalDefaultX = defaultX + notDefaultX
plotX = [0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]

count_005 = 0
count_015 = 0
count_025 = 0
count_035 = 0
count_045 = 0
count_055 = 0
count_065 = 0
count_075 = 0
count_085 = 0
count_095 = 0
for i in totalDefaultX:
    if 0 <= i < 0.1:
        count_005 += 1
    elif 0.1 <= i < 0.2:
        count_015 +=1
    elif 0.2 <= i < 0.3:
        count_025 +=1
    elif 0.3 <= i < 0.4:
        count_035 +=1
    elif 0.4 <= i < 0.5:
        count_045 +=1
    elif 0.5 <= i < 0.6:
        count_055 +=1
    elif 0.6 <= i < 0.7:
        count_065 +=1
    elif 0.7 <= i < 0.8:
        count_075 +=1
    elif 0.8 <= i <0.9:
        count_085 +=1
    elif 0.9 <= i <= 1.0:
        count_095 +=1
plotY = [count_005,count_015,count_025,count_035,count_045,count_055,count_065,count_075,count_085,count_095]

count_default_005 = 0
count_default_015 = 0
count_default_025 = 0
count_default_035 = 0
count_default_045 = 0
count_default_055 = 0
count_default_065 = 0
count_default_075 = 0
count_default_085 = 0
count_default_095 = 0
for i in defaultX:
    if 0 <= i < 0.1:
        count_default_005 += 1
    elif 0.1 <= i < 0.2:
        count_default_015 +=1
    elif 0.2 <= i < 0.3:
        count_default_025 +=1
    elif 0.3 <= i < 0.4:
        count_default_035 +=1
    elif 0.4 <= i < 0.5:
        count_default_045 +=1
    elif 0.5 <= i < 0.6:
        count_default_055 +=1
    elif 0.6 <= i < 0.7:
        count_default_065 +=1
    elif 0.7 <= i < 0.8:
        count_default_075 +=1
    elif 0.8 <= i <0.9:
        count_default_085 +=1
    elif 0.9 <= i <= 1.0:
        count_default_095 +=1
plotDefaultY = [count_default_005/count_005,count_default_015/count_015,count_default_025/count_025,
                count_default_035/count_035,count_default_045/count_045,count_default_055/count_055,
                count_default_065/count_065,count_default_075/count_075,count_default_085/count_085,
                count_default_095/count_095]


# Start to plot with Tushare Data
fig = plt.figure(figsize=(30,20))

ax1 = fig.add_subplot(111)
ax1.bar(plotX, plotY, width=0.05, edgecolor='black')
ax1.set_ylabel('Number of Issuers',size=20)
ax1.set_xlabel('Default probability predictions',size=20)
plt.yticks(fontsize=20)
plt.xticks(np.arange(0,1,0.1), fontsize=20)

ax2 = ax1.twinx()
ax2.plot(plotX, plotDefaultY, color='red', label='Default rate')
ax2.set_ylabel('Actual default rate (%)',size=20)
plt.yticks(fontsize=20)
plt.title('KMV - Bond Issuer Default Rate Prediction',fontsize=20)
plt.legend(loc='best',fontsize=20)

plt.show()
fig.savefig('CSMARBar.jpg', dpi = 750)
os.chdir('../../')

# ROC and AUROC

In [ ]:
# First, set cutoff values
cutoff = np.arange(0,1.001,0.001)

In [ ]:
tushareMergeResult = tushareMergeResult[tushareMergeResult['default_probability'] != '-']
afterDeleteNa = tushareMergeResult.dropna(axis=0,how='any')

In [ ]:
# Then, build Confusion Matrix
tpr = []
fpr = []
for i in cutoff:
    tp = afterDeleteNa.loc[(afterDeleteNa['default_probability'] >= i) & (afterDeleteNa['default_judgment'] == 1)]
    fn = afterDeleteNa.loc[(afterDeleteNa['default_probability'] < i) & (afterDeleteNa['default_judgment'] == 1)]
    fp = afterDeleteNa.loc[(afterDeleteNa['default_probability'] >= i) & (afterDeleteNa['default_judgment'] == 0)]
    tn = afterDeleteNa.loc[(afterDeleteNa['default_probability'] < i) & (afterDeleteNa['default_judgment'] == 0)]    
    
    tpr.append(tp.shape[0]/(tp.shape[0]+fn.shape[0]))
    fpr.append(fp.shape[0]/(fp.shape[0]+tn.shape[0]))

In [ ]:
# Then plot ROC
fig = plt.figure(figsize=(20,15))
plt.plot(fpr,tpr,linestyle='-',color='green', label='Predicted ROC Curve',linewidth=2)
plt.plot([0, 1], [0, 1], color="gray", linestyle="--", label="Random ROC Curve",linewidth=2)
plt.xlabel('False Positive Rate',fontsize=20)
plt.ylabel('True Positive Rate',fontsize=20)
plt.yticks(np.arange(0,1.1,0.1), fontsize=20)
plt.xticks(np.arange(0,1.1,0.1), fontsize=20)
# Ideal Curve
best_point = [afterDeleteNa[afterDeleteNa['default_judgment'] == 1].shape[0] / afterDeleteNa.shape[0], 1]
plt.plot([0, best_point[0], 1], [0, best_point[1], 1], color="red", label="Ideal ROC Curve", zorder=10,linewidth=2)
# Fill in Color
plt.fill_between(fpr, tpr, fpr, color="yellow", alpha=0.3)
plt.fill_between(fpr, [1 if i * afterDeleteNa.shape[0] / afterDeleteNa[afterDeleteNa['default_judgment'] == 1].shape[0] >= 1 else i * afterDeleteNa.shape[0] / afterDeleteNa[afterDeleteNa['default_judgment'] == 1].shape[0] for i in fpr], tpr, color="gray", alpha=0.3)

plt.title('Receiver Operating Characteristic Curve',fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()
os.chdir('Default_Data/Plot/')
fig.savefig('ROC.jpg', dpi = 750)
os.chdir('../../')


In [ ]:
# Calculate AUROC
from sklearn.metrics import roc_auc_score
afterDeleteNa = tushareMergeResult.dropna(axis=0,how='any')
y_truth = np.array(afterDeleteNa['default_judgment'].tolist())
y_pred = np.array(afterDeleteNa['default_probability'].tolist())
auc_score = roc_auc_score(y_truth,y_pred)
auc_score

# CAP and AR

In [ ]:
# Also set cutoff
cutoff = np.arange(0,1.001,0.001)
tushareMergeResult = tushareMergeResult[tushareMergeResult['default_probability'] != '-']

In [ ]:
ctr = []
cdr = []
gini_together_1 = []
gini_together_2 = []
for i in cutoff:
    tr = afterDeleteNa.loc[afterDeleteNa['default_probability'] >= i]
    dr = afterDeleteNa.loc[(afterDeleteNa['default_probability'] >= i) & (afterDeleteNa['default_judgment'] == 1)]
    dt = afterDeleteNa.loc[afterDeleteNa['default_judgment'] == 1]
    ctr.append(tr.shape[0]/afterDeleteNa.shape[0])
    cdr.append(dr.shape[0]/dt.shape[0])
    gini_together_1.append([tr.shape[0]/afterDeleteNa.shape[0], dr.shape[0]/dt.shape[0]])
    gini_together_2.append([dr.shape[0]/dt.shape[0], tr.shape[0]/afterDeleteNa.shape[0]])

In [ ]:
ar_score = 2*auc_score - 1
ar_score

In [ ]:
import numpy as np

def gini(data_list):
    data_length = len(data_list)
    total_sum = np.sum(data_list)
    total_gini = 0
    for i in range(data_length):
        temp_denominator = data_list[i][0] + data_list[i][1]
        temp_son = np.sum(data_list[i])
        temp_gini = 1 - np.power(data_list[i][0]/temp_denominator, 2) - np.power(data_list[i][1]/temp_denominator, 2)
        total_gini = temp_gini * temp_son / total_sum + total_gini
    return total_gini
print(gini(gini_together_1))
print(gini(gini_together_2))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams["font.sans-serif"] = ["SimHei"]

def cap_plot(predictions, labels, cut_point=100000):
    sample_size = len(labels)
    bad_label_size = len([i for i in labels if i == 1])
    score_thres = np.linspace(1, 0, cut_point)
    x_list = []
    y_list = []
    for thres in score_thres:
        x = len([i for i in predictions if i >= thres])
        x_list.append(x / sample_size)
        y = len([(i, j) for i, j in zip(predictions, labels) if i >= thres and j == 1])
        y_list.append(y / bad_label_size)

    # Real Curve
    fig = plt.figure(figsize=(30,20))
    plt.plot(x_list, y_list, color="green", label="Predicted Curve",linewidth=2)
    plt.yticks(np.arange(0,1.1,0.1),fontsize=20)
    plt.xticks(np.arange(0,1.1,0.1),fontsize=20)
    
    # Ideal Curve
    best_point = [bad_label_size / sample_size, 1]
    plt.plot([0, best_point[0], 1], [0, best_point[1], 1], color="red", label="Ideal Curve", zorder=10,linewidth=2)
    # Add the ideal point's axes
    #plt.scatter(best_point[0], 1, color="white", edgecolors="red", s=30, zorder=30)
    #plt.text(best_point[0] + 0.1, 0.95, "{}/{},{}".format(bad_label_size, sample_size, 1), ha="center", va="center")
    
    # Random Curve
    plt.plot([0, 1], [0, 1], color="gray", linestyle="--", label="Random Curve",linewidth=2)
    
    # Fill in Color
    plt.fill_between(x_list, y_list, x_list, color="yellow", alpha=0.3)
    plt.fill_between(x_list, [1 if i * sample_size / bad_label_size >= 1 else i * sample_size / bad_label_size for i in x_list], y_list, color="gray", alpha=0.3)

    # Calculate AR Value
    # Square under Real Curve
    actual_area = np.trapz(y_list, x_list) - 1 * 1 / 2
    best_area = 1 * 1 / 2 - 1 * bad_label_size / sample_size / 2
    ar_value = actual_area / best_area
    plt.title("Cumulative Accuracy Profile Curve (AR={:.4f})".format(ar_value), fontsize=20)
    plt.xlabel('Cumulative proportion of real default in total universe', fontsize=20)
    plt.ylabel('Cumulative proportion of real default in default universe', fontsize=20)

    plt.legend(loc=4, fontsize=20)
    plt.grid()
    plt.show()
    os.chdir('Default_Data/Plot/')
    fig.savefig('CAP.jpg', dpi = 750)
    os.chdir('../../')

if __name__ == "__main__":
    cap_plot(afterDeleteNa['default_probability'], afterDeleteNa['default_judgment'])

# K-S

In [ ]:
# Then, build Confusion Matrix
cutoff = np.arange(0,1.001,0.001)
tpr = []
fpr = []
for i in cutoff:
    tp = tushareMergeResult.loc[(tushareMergeResult['default_probability'] >= i) & (tushareMergeResult['default_judgment'] == 1)]
    fn = tushareMergeResult.loc[(tushareMergeResult['default_probability'] < i) & (tushareMergeResult['default_judgment'] == 1)]
    fp = tushareMergeResult.loc[(tushareMergeResult['default_probability'] >= i) & (tushareMergeResult['default_judgment'] == 0)]
    tn = tushareMergeResult.loc[(tushareMergeResult['default_probability'] < i) & (tushareMergeResult['default_judgment'] == 0)]    
    
    tpr.append(tp.shape[0]/(tp.shape[0]+fn.shape[0]))
    fpr.append(fp.shape[0]/(fp.shape[0]+tn.shape[0]))


In [ ]:
# Then plot K-S
fig = plt.figure(figsize=(20,15))
ks = np.array(tpr) - np.array(fpr)
print(ks.max())
plt.plot(cutoff,tpr,linestyle='-',color='r', label='TPR',linewidth=2)
plt.plot(cutoff,fpr,linestyle='-',color='green', label='FPR',linewidth=2)
plt.vlines(cutoff[list(ks).index(ks.max())], fpr[list(ks).index(ks.max())], tpr[list(ks).index(ks.max())], color='blue', linestyle='--',linewidth=2)
plt.xlabel('Cutoff',fontsize=20)
plt.ylabel('Percent population',fontsize=20)

plt.yticks(np.arange(0,1.1,0.1), fontsize=20)
plt.xticks(np.arange(0,1.1,0.1), fontsize=20)
plt.title('K-S Curve',fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()
os.chdir('Default_Data/Plot/')
fig.savefig('KS.jpg', dpi = 750)
os.chdir('../../')

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.metrics import roc_curve
from scipy.stats import ks_2samp
def ks_calc_cross(data,pred,y_label):
    crossfreq = pd.crosstab(data[pred[0]],data[y_label[0]])
    crossdens = crossfreq.cumsum(axis=0) / crossfreq.sum()
    crossdens['gap'] = abs(crossdens[0] - crossdens[1])
    ks = crossdens[crossdens['gap'] == crossdens['gap'].max()]
    return ks,crossdens

def ks_calc_auc(data,pred,y_label):
    fpr,tpr,thresholds= roc_curve(data[y_label[0]],data[pred[0]])
    ks = max(tpr-fpr)
    return ks

def ks_calc_2samp(data,pred,y_label):
    Bad = data.loc[data[y_label[0]]==1,pred[0]]
    Good = data.loc[data[y_label[0]]==0, pred[0]]
    data1 = Bad.values
    data2 = Good.values
    n1 = data1.shape[0]
    n2 = data2.shape[0]
    data1 = np.sort(data1)
    data2 = np.sort(data2)
    data_all = np.concatenate([data1,data2])
    cdf1 = np.searchsorted(data1,data_all,side='right')/(1.0*n1)
    cdf2 = (np.searchsorted(data2,data_all,side='right'))/(1.0*n2)
    ks = np.max(np.absolute(cdf1-cdf2))
    cdf1_df = pd.DataFrame(cdf1)
    cdf2_df = pd.DataFrame(cdf2)
    cdf_df = pd.concat([cdf1_df,cdf2_df],axis = 1)
    cdf_df.columns = ['cdf_Bad','cdf_Good']
    cdf_df['gap'] = cdf_df['cdf_Bad']-cdf_df['cdf_Good']
    return ks,cdf_df

data = afterDeleteNa[['default_probability','default_judgment']]
ks1,crossdens=ks_calc_cross(data,['default_probability'], ['default_judgment'])
ks2=ks_calc_auc(data,['default_probability'], ['default_judgment'])
ks3=ks_calc_2samp(data,['default_probability'], ['default_judgment'])
get_ks = lambda y_pred,y_true: ks_2samp(y_pred[y_true==1], y_pred[y_true!=1]).statistic
ks4=get_ks(data['default_probability'],data['default_judgment'])
print('KS1:',ks1['gap'].values)
print('KS2:',ks2)
print('KS3:',ks3[0])
print('KS4:',ks4)